# PROJECT: Pet Classification Tensorflow Model Using CNN

# Project Objective
To build a CNN model that classifies the given pet images correctly into dog and cat images.

The project scope document specifies the requirements for the project “Pet Classification Model Using CNN.” Apart from specifying the functional and nonfunctional requirements for the project, it also serves as an input for project scoping.

# Project Description and Scope
We are provided with the following resources that can be used as inputs for your model:

A collection of images of pets, that is​, ​cats and dogs. These images are of different sizes with varied lighting conditions.
Code template containing the following code blocks: a. Import modules (part 1) b. Set hyper parameters (part 2) c. Read image data set (part 3) d. Run TensorFlow model (part 4) You are expected to write the code for CNN image classification model (between Parts 3 and 4) using TensorFlow that trains on the data and calculates the accuracy score on the test data.

# Project Guidelines
Begin by extracting ipynb file and the data in the same folder. The CNN model (cnn_model_fn) should have the following layers: ● Input layer ● Convolutional layer 1 with 32 filters of kernel size[5,5] ● Pooling layer 1 with pool size**[2,2] **and stride 2 ● Convolutional layer 2 with 64 filters of kernel size[5,5] ● Pooling layer 2 with pool size[2,2] and stride 2 ● Dense layer whose output size is fixed in the hyper parameter: fc_size=32 ● Dropout layer with dropout probability 0.4 Predict the class by doing a softmax on the output of the dropout lay bold text

This should be followed by training and evaluation: For the training step, define the loss function and minimize it ● For the evaluation step, calculate the accuracy Run the program for 100, 200, and 300 iterations, respectively. Follow this by a report on the final accuracy and loss on the evaluation data. Prerequisites To execute this project, refer to the installation guide in the downloads section of LMS.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm

In [3]:
TRAIN_DIR = '/content/drive/MyDrive/PETDB/train'
TEST_DIR = '/content/drive/MyDrive/PETDB/test'
IMG_SIZE = 224
LR = 1e-3

MODEL_NAME = 'pet_classifier_2_Conv_basic'

In [4]:
def label_img(folder_name):
    
    # conversion to one-hot array [cat,dog]
    #                            [much cat, no dog]
    if folder_name == 'cats': return [1,0]
    #                             [no cat, very doggo]
    elif folder_name == 'dogs': return [0,1]

In [5]:
def create_train_data():
    training_data = []
    for n,folder in enumerate(os.listdir(TRAIN_DIR)):
      images = os.listdir(os.path.join(TRAIN_DIR,folder))
      for i,image in enumerate(images):
        label = label_img(folder)
        path = os.path.join(TRAIN_DIR,folder,image)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [6]:
X = create_train_data()

/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [7]:
os.getcwd()

'/content'

In [8]:
def create_test_data():
    test_data = []
    for n,folder in enumerate(os.listdir(TEST_DIR)):
      images = os.listdir(os.path.join(TEST_DIR,folder))
      for i,image in enumerate(images):
        label = label_img(folder)
        path = os.path.join(TEST_DIR,folder,image)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        img = img/255
        test_data.append([np.array(img),np.array(label)])
    shuffle(test_data)
    np.save('test_data.npy', test_data)
    return test_data

In [9]:
Y = create_test_data()

/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


# Model Specification
The CNN model (cnn_model_fn) should have the following layers:

*   Input layer 
*  Convolutional layer 1 with 32 filters of kernel size[5,5]
* Pooling layer 1 with pool size**[2,2] **and stride 2
* Convolutional layer 2 with 64 filters of kernel size[5,5] 
* Pooling layer 2 with pool size[2,2] and stride 2 
* Dense layer whose output size is fixed in the hyper parameter: fc_size=32
* Dropout layer with dropout probability 0.4 Predict the class by doing a softmax on the output of the dropout lay bold text




In [10]:
!pip install tflearn 

In [11]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

#Input layer
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

#Convolutional layer 1 with 32 filters of kernel size[5,5]
convnet = conv_2d(convnet, 32, 5, activation='relu')

#Pooling layer 1 with pool size**[2,2] **and stride 2
convnet = max_pool_2d(convnet, 2,strides = 2)

#Convolutional layer 2 with 64 filters of kernel size[5,5]
convnet = conv_2d(convnet, 64, 5, activation='relu')

#Pooling layer 2 with pool size[2,2] and stride 2
convnet = max_pool_2d(convnet, 2,strides = 2)

#Dense layer whose output size is fixed in the hyper parameter: fc_size=32
convnet = fully_connected(convnet, 32, activation='relu')

#Dropout layer with dropout probability 0.4 Predict the class by doing a softmax on the output of the dropout lay bold text
convnet = dropout(convnet, 0.4)
convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
model = tflearn.DNN(convnet, tensorboard_dir='log')


Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
train = X
test = Y

In [13]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

In [14]:
history= model.fit({'input': X}, {'targets': Y}, n_epoch=300, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

---------------------------------
Run id: pet_classifier_2_Conv_basic
Log directory: log/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 40
Validation samples: 20
--
Training Step: 1  | time: 4.297s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 | val_loss: 4.49477 - val_acc: 0.5000 -- iter: 40/40
--
Training Step: 2  | total loss: 9.66164 | time: 1.123s
| Adam | epoch: 002 | loss: 9.66164 - acc: 0.4500 | val_loss: 8.21560 - val_acc: 0.5000 -- iter: 40/40
--
Training Step: 3  | total loss: 11.17633 | time: 1.116s
| Adam | epoch: 003 | loss: 11.17633 - acc: 0.4909 | val_loss: 10.84616 - val_acc: 0.5000 -- iter: 40/40
--
Training Step: 4  | total loss: 11.42878 | time: 1.116s
| Adam | epoch: 004 | loss: 11.42878 - acc: 0.4977 | val_loss: 11.51293 - val_acc: 0.5000 -- iter: 40/40
--
Training Step: 5  | total loss: 11.48703 | time: 1.119s
| Adam | epoch: 005 | loss: 11.48703 - acc: 0.4993 | val_

In [18]:
history

In [15]:
model.save('/content/drive/MyDrive/PETDB/{}'.format(MODEL_NAME))

INFO:tensorflow:/content/drive/MyDrive/PETDB/pet_classifier_2_Conv_basic is not in all_model_checkpoint_paths. Manually adding it.
